# Task 2: Smart Route Planner (Travelling Salesman Problem)

This notebook implements and visualizes a solution to the Travelling Salesman Problem (TSP) for a delivery robot using a Greedy (Nearest Neighbor) approach.

## 1. Setup and Imports

Import necessary libraries: `numpy` for numerical operations, `matplotlib` for plotting and animation, and `math` for distance calculations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
import random
from IPython.display import HTML, display

## 2. Define Locations and Distance Function

Define the coordinates for the warehouse (start/end point) and the delivery locations. Also, define a function to calculate the Euclidean distance between two points.

In [ ]:
# Warehouse (index 0) and delivery locations
locations = {
    'Warehouse': (2, 2),
    'A': (1, 5),
    'B': (4, 8),
    'C': (7, 6),
    'D': (9, 3),
    'E': (5, 1)
}

location_names = list(locations.keys())
coords = np.array(list(locations.values()))
num_locations = len(locations)
warehouse_index = location_names.index('Warehouse') # Should be 0

def calculate_distance(point1, point2):
    """Calculates Euclidean distance between two points."""
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

## 3. TSP Algorithm: Greedy (Nearest Neighbor)

Implement the Nearest Neighbor algorithm. Starting from the warehouse, the robot repeatedly visits the nearest unvisited location until all locations are visited, then returns to the warehouse.

In [ ]:
def nearest_neighbor_tsp(coords, start_node_index=0):
    """Solves TSP using the Nearest Neighbor heuristic."""
    num_nodes = len(coords)
    unvisited = list(range(num_nodes))
    current_node = start_node_index
    route = [current_node]
    unvisited.remove(current_node)
    total_distance = 0

    while unvisited:
        nearest_node = -1
        min_dist = float('inf')

        for node in unvisited:
            dist = calculate_distance(coords[current_node], coords[node])
            if dist < min_dist:
                min_dist = dist
                nearest_node = node
        
        total_distance += min_dist
        current_node = nearest_node
        route.append(current_node)
        unvisited.remove(current_node)

    # Return to start
    total_distance += calculate_distance(coords[current_node], coords[start_node_index])
    route.append(start_node_index)

    return route, total_distance

## 4. Visualization Functions

Create functions to plot the locations and the final route, and to animate the robot's movement along the route.

In [ ]:
def plot_route(coords, route, location_names, total_distance, title='TSP Route'):
    """Plots the locations and the calculated TSP route."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Plot route segments
    route_coords = coords[route]
    ax.plot(route_coords[:, 0], route_coords[:, 1], 'o-', label='Robot Path', markersize=8, color='blue', zorder=1)

    # Plot locations
    ax.scatter(coords[:, 0], coords[:, 1], s=100, color='red', zorder=2) # All locations
    ax.scatter(coords[warehouse_index, 0], coords[warehouse_index, 1], s=150, color='green', marker='s', label='Warehouse', zorder=3) # Warehouse

    # Annotate locations
    for i, name in enumerate(location_names):
        ax.text(coords[i, 0] + 0.1, coords[i, 1] + 0.1, name, fontsize=12)

    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f'{title}\nTotal Distance: {total_distance:.2f}')
    ax.legend()
    ax.grid(True)
    ax.axis('equal') # Ensure aspect ratio is equal
    plt.show()

def animate_route(coords, route, location_names, total_distance, title='TSP Route Animation'):
    """Animates the robot moving along the TSP route."""
    fig, ax = plt.subplots(figsize=(8, 8))
    route_coords = coords[route]

    # Plot locations static background
    ax.scatter(coords[:, 0], coords[:, 1], s=100, color='red', zorder=2)
    ax.scatter(coords[warehouse_index, 0], coords[warehouse_index, 1], s=150, color='green', marker='s', label='Warehouse', zorder=3)
    for i, name in enumerate(location_names):
        ax.text(coords[i, 0] + 0.1, coords[i, 1] + 0.1, name, fontsize=12)

    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f'{title}\nTotal Distance: {total_distance:.2f}')
    ax.legend()
    ax.grid(True)
    ax.axis('equal')

    # Line object to update for the path
    line, = ax.plot([], [], 'o-', color='blue', markersize=8, label='Robot Path')
    # Robot position marker
    robot, = ax.plot([], [], 'X', color='purple', markersize=12, label='Robot')

    # Animation update function
    def update(frame):
        # Draw path segments up to the current frame
        current_path_coords = route_coords[:frame+1]
        line.set_data(current_path_coords[:, 0], current_path_coords[:, 1])
        
        # Update robot position
        if frame < len(route_coords):
             robot.set_data(route_coords[frame, 0], route_coords[frame, 1])
        else: # Keep robot at the end position (warehouse)
             robot.set_data(route_coords[-1, 0], route_coords[-1, 1])
             
        # Update legend dynamically (optional, can be complex)
        # handles, labels = ax.get_legend_handles_labels()
        # ax.legend(handles=handles, labels=labels)
        
        return line, robot

    # Create animation
    # Frames go from 0 to num_locations (inclusive) to show all segments + final robot position
    ani = animation.FuncAnimation(fig, update, frames=len(route), interval=500, blit=True, repeat=False)
    plt.close(fig) # Prevent duplicate static plot
    return ani

## 5. Run TSP Solver and Visualize

Execute the Nearest Neighbor algorithm and display the results.

In [ ]:
print("Running Nearest Neighbor TSP...")
nn_route, nn_distance = nearest_neighbor_tsp(coords, warehouse_index)
print(f"Route found: {' -> '.join([location_names[i] for i in nn_route])}")
print(f"Total distance: {nn_distance:.2f}")

# Static Plot
plot_route(coords, nn_route, location_names, nn_distance, title='Nearest Neighbor TSP Route')

# Animation
print("Generating animation...")
ani_nn = animate_route(coords, nn_route, location_names, nn_distance, title='Nearest Neighbor TSP Animation')
display(HTML(ani_nn.to_jshtml()))
# To save animation (optional):
# ani_nn.save('tsp_nn_animation.gif', writer='pillow', fps=2)
# print("Animation saved as tsp_nn_animation.gif")

## 6. Explanation

### Approach Used

1.  **Problem Definition**: The task is modeled as the Travelling Salesman Problem (TSP), where the goal is to find the shortest possible route that visits each delivery location exactly once and returns to the starting warehouse.
2.  **Algorithm**: The **Nearest Neighbor (NN)** algorithm, a greedy heuristic, was implemented.
    *   It starts at the designated warehouse.
    *   At each step, it travels to the closest unvisited location.
    *   This process repeats until all locations have been visited.
    *   Finally, it returns to the starting warehouse.
3.  **Distance Calculation**: Euclidean distance is used to measure the distance between locations based on their 2D coordinates.
4.  **Visualization**: 
    *   A static plot shows all locations, the warehouse, and the final calculated route with the total distance.
    *   An animation visualizes the robot moving sequentially through the locations according to the NN route.

### Pros and Cons of Nearest Neighbor

*   **Pros**:
    *   Simple and intuitive to understand and implement.
    *   Computationally fast, especially compared to exact methods like brute force or dynamic programming, making it suitable for quick approximations.
*   **Cons**:
    *   **Not Optimal**: It's a greedy heuristic and does not guarantee finding the absolute shortest route. The choice made at each step (going to the *nearest* neighbor) might lead to poor choices later in the tour.
    *   **Sensitivity to Starting Point**: The resulting route can vary depending on the starting location.

### Assumptions Made

1.  **Euclidean Distance**: Travel cost is directly proportional to the straight-line (Euclidean) distance between locations.
2.  **Symmetric Costs**: The cost of travel between two points (A to B) is the same as the cost in the reverse direction (B to A).
3.  **Complete Graph**: It's possible to travel directly between any two locations.
4.  **Static Locations**: The positions of the warehouse and delivery points do not change.
5.  **Robot Capabilities**: The robot can instantly travel between points (travel time is not explicitly modeled beyond distance).